In [1]:
# Import all necessary packages and functions here
# Make sure before running this, your selected python environment actually has all required packages.
from bs4 import BeautifulSoup
import configparser
import requests
import json
import pickle
import pandas as pd
import datetime
from constants_funcs import get_session, int_or_float_or_str
with open('constants.json', 'r') as d:
    constants = json.load(d) # check out this file to see all your options, and feel free to add more.

In [2]:
# First, open the smaller files not in the .zip. JSON files opened normally, csv's opened with Pandas.
with open('./pp_by_username.json', 'r') as fp:
    id_lookup_by_username = json.load(fp)
full_question_history = pd.read_csv('./FULL_QUESTION_HISTORY_v2.csv', header=0)
player_profiles = pd.read_csv('./player_profiles.csv', header=0)

In [3]:
# Unzip the archive.zip file, then set the path variable below. If it's in the same folder, set path = './'.
path = './FULL_DATA' # mine happens to be in the FULL_DATA folder in my repo-change as needed.
leagues_by_season = pd.read_csv(f'{path}/leagues.csv') # for more info on f-formatted strings, see: https://realpython.com/python-f-strings/#f-strings-a-new-and-improved-way-to-format-strings-in-python
rundles_by_season = pd.read_csv(f'{path}/rundles.csv')
matches_history = pd.read_csv(f'{path}/matches_history.csv')
match_participation = pd.read_csv(f'{path}/match_participation.csv')
player_answer_history = pd.read_csv(f'{path}/player_answer_history.csv')
# Note that this may take a hot minute, considering the size of these (especially player_answer_history at 3.6 GB!)
# DtypeWarnings are OK for now, may be resolved in future versions.

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
grp = player_answer_history.sort_values('qnum', ascending=True).groupby(['season', 'md', 'player_id'])
grp.head()

,Unnamed: 0,match_id,rundle,league,div,season,md,qnum,player,correct,def_given,player_id,id
0,0,25904701.0,A,League,0,30,1,1,white,True,2.0,19962,NaN
15485173,15485173,1895799.0,E,Metro,2,76,6,1,cohenbc,False,0.0,14158,NaN
48837937,48837937,4763601.0,E,Sakura,1,89,24,1,50312,False,2.0,50312,NaN
48837936,48837936,4763601.0,E,Sakura,1,89,24,1,12716,True,3.0,12716,NaN
48837925,48837925,4763600.0,E,Sakura,1,89,24,1,16551,False,2.0,16551,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27973077,27973077,2874257.0,E,Pacific,2,82,7,5,fowlers,False,3.0,5764,NaN
17877368,17877368,2088523.0,D,Canyon,2,77,19,5,patorak,True,2.0,29761,NaN
17880068,17880068,2090104.0,E,Garden,2,77,19,5,flynnp,False,1.0,2248,NaN
32458953,32458953,3261166.0,E,Plaza,1,84,6,5,27751,True,2.0,27751,NaN


In [10]:
player_answer_history['corr_string'] = grp.correct.transform(lambda x: ','.join(str(x)))
player_answer_history.head()

,Unnamed: 0,match_id,rundle,league,div,season,md,qnum,player,correct,def_given,player_id,id,corr_string
0,0,25904701.0,A,League,0,30,1,1,white,True,2.0,19962,NaN,"0, , , , , ,T,r,u,e,\n,1, , , , ,F,a,l,s,e,\n,..."
1,1,25904701.0,A,League,0,30,1,2,white,False,3.0,19962,NaN,"0, , , , , ,T,r,u,e,\n,1, , , , ,F,a,l,s,e,\n,..."
2,2,25904701.0,A,League,0,30,1,3,white,True,1.0,19962,NaN,"0, , , , , ,T,r,u,e,\n,1, , , , ,F,a,l,s,e,\n,..."
3,3,25904701.0,A,League,0,30,1,4,white,False,1.0,19962,NaN,"0, , , , , ,T,r,u,e,\n,1, , , , ,F,a,l,s,e,\n,..."
4,4,25904701.0,A,League,0,30,1,5,white,False,2.0,19962,NaN,"0, , , , , ,T,r,u,e,\n,1, , , , ,F,a,l,s,e,\n,..."


In [21]:
player_answer_history['def_string'] = player_answer_history[
    ['season', 'md', 'player_id', 'qnum', 'def_given']].sort_values(
    'qnum', ascending=True).groupby(['season', 'md', 'player_id'])['def_given'].transform(
    lambda x: ','.join(str(x))
    )

In [141]:
def def_string(val):
    return 'FFFFFF' if 'N,a,N' in val else ''.join(map(
        lambda x: x[-6],
        val.split('\n')[:6]))
def corr_string(val):
    return 'FFFFFF' if 'N,a,N' in val else ''.join(map(
        lambda x: str(int('T,r,u,e' in x)),
        val.split('\n')[:6]))

In [106]:
reduced_pah = player_answer_history[player_answer_history.qnum == 1] # only need one row per MD.
reduced_pah['true_corr'] = reduced_pah.apply(lambda row: corr_string(row.corr_string), axis=1)
print('done true corr')
reduced_pah['true_def'] = reduced_pah.apply(lambda row: def_string(row.def_string), axis=1)

<ipython-input-106-4f916181ffc3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_pah['true_corr'] = reduced_pah.apply(lambda row: corr_string(row.corr_string), axis=1)


done true corr


<ipython-input-106-4f916181ffc3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_pah['true_def'] = reduced_pah.apply(lambda row: def_string(row.def_string), axis=1)


In [139]:
print(reduced_pah.shape)
reduced_pah.head() # dropped some columns to get to this view.

(11580806, 9)


,match_id,rundle,league,div,season,md,player_id,true_corr,true_def
0,25904701.0,A,League,0,30,1,19962,101001,231120
6,25904701.0,A,League,0,30,1,28118,011101,021123
12,4305401.0,A,League,0,30,1,19713,111011,201213
18,4305401.0,A,League,0,30,1,27800,010010,012321
24,2302401.0,A,League,0,30,1,20858,111001,232110


In [140]:
reduced_pah.to_csv('./FULL_DATA/match_participation_V2.csv', index=False)